In [1]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

df = df [:1000]
vectorizer = CountVectorizer(binary=True)
text_data = df["Phrase"]
text_data = vectorizer.fit_transform(text_data)

import torch

text_tensor = torch.from_numpy(text_data.todense()).float()

#ajout du cas où on ne connait pas le mot
voca = vectorizer.get_feature_names() + list(['<unk>'])
#ajout d'un zéro à la fin de chaque tensor
unk = torch.zeros([text_tensor.shape[0],1], dtype=torch.float)
text_tensor = torch.cat((text_tensor, unk), 1)

TypeError: from_numpy() takes no keyword arguments

In [3]:
vectorizer = CountVectorizer(binary=True)
label_data = pd.get_dummies(df["Sentiment"])

label_tensor = torch.tensor(label_data.values, dtype=torch.long)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input.view(1, len(voca)), hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size, dtype=torch.float)


In [55]:
input_dim = len(voca)
hidden_dim = 100
output_dim = 5

rnn = RNN(input_dim, hidden_dim, output_dim)

In [56]:
# hidden = torch.zeros(1000, hidden_dim, dtype=torch.float)

# output, next_hidden = rnn(text_tensor, hidden)

In [57]:
criterion = nn.NLLLoss()

In [109]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()
    
    print(rnn.parameters()[0].grad)
#     print(line_tensor.size(), hidden.size())
#     for i in range(line_tensor.size()[0]):    
#         print(line_tensor[i].size())
    output, hidden = rnn(line_tensor, hidden)

    loss = criterion(output, torch.tensor([category_tensor]))
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        print(type(p.grad))
        p.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [110]:
current_loss = 0

# for label, text in zip(label_tensor, text_tensor):
for i in range(text_tensor.size()[0]):
    output, loss = train(df["Sentiment"][i], text_tensor[i])
    current_loss += loss

TypeError: 'generator' object is not subscriptable